In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
import random
from sklearn.metrics import mean_absolute_error


In [71]:
v_train = pd.read_csv('dados_treino_volume_com_valor_anterior.csv')
v_test = pd.read_csv('dados_teste_volume_com_valor_anterior.csv')

In [72]:
df_remove = v_train.loc[(v_train['date'] >= '2016-10-01') 
                         & (v_train['date'] <= '2016-10-07') 
                        ]

v_train = v_train.drop(df_remove.index)

In [83]:
df_train_list = [v_train,]
random.shuffle(df_train_list)
df_train = pd.concat(df_train_list)

df_train_grouped = df_train.groupby(["tollgate_id", "direction"])
df_test_grouped = v_test.groupby(["tollgate_id", "direction"])
result = []
oob = []
for key, train_data in df_train_grouped:

    test_data = df_test_grouped.get_group(key)
    len_train = len(train_data)
    train_data = train_data.append(test_data)[train_data.columns.tolist()]


    #train_data = feature_transform_knn(key, train_data)

    regressor_cubic = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=10, oob_score=True)

    train_data = pd.DataFrame.reset_index(train_data)
    train_data = train_data.drop("index", axis=1)
    y = train_data.ix[:len_train - 1, :]["volume"]


    x = train_data.ix[:len_train - 1, 5:]
    x1 = train_data.ix[len_train:, 5:]

    regressor_cubic.fit(x, y)
    yhat = regressor_cubic.predict(x1)

    test_data["volume"] = yhat
    result.append(test_data[['tollgate_id', 'time_window', 'direction', 'volume']])


df_result = pd.concat(result, axis=0)

/home/matheuspds/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/

ValueError: Number of features of the model must match the input. Model n_features is 6 and input n_features is 7 

In [74]:
df_result.head()

,tollgate_id,time_window,direction,volume
0,1,"[06:00:00,06:20:00)",0,12.484
5,1,"[06:20:00,06:40:00)",0,16.200
10,1,"[06:40:00,07:00:00)",0,19.844
15,1,"[07:00:00,07:20:00)",0,26.156
20,1,"[07:20:00,07:40:00)",0,28.204


In [75]:
test_0 = pd.read_csv("predicao-com-serie/data_after_process/test_0.csv")

In [76]:
df_remove = test_0.loc[(test_0['volume'] == 0)]

test_0 = test_0.drop(df_remove.index)
test_0.head()

,time_start,volume,direction,tollgate_id
18,2016-10-25 06:00:00,10,0,1
19,2016-10-25 06:20:00,17,0,1
20,2016-10-25 06:40:00,30,0,1
21,2016-10-25 07:00:00,25,0,1
22,2016-10-25 07:20:00,38,0,1


In [77]:
volume_real = test_0['volume'].values
volume_prediction = df_result['volume'].values

In [78]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [79]:
mean_absolute_percentage_error(volume_real, volume_prediction)

50.85381247751025

In [80]:
mean_absolute_error(volume_real, volume_prediction)

23.03997777777778

In [81]:
volume_real

array([ 10,  17,  30,  25,  38,  44,  52,  44,  46,  50,  40,  50,  12,
         9,  25,  31,  32,  45,  38,  36,  44,  49,  45,  46,  14,  14,
        23,  20,  36,  42,  40,  37,  45,  49,  41,  45,   8,  21,  14,
        28,  37,  46,  62,  48,  57,  78,  67,  58,  11,  17,  19,  27,
        37,  24,  60,  48,  52,  48,  62,  48,  18,  25,  30,  49,  40,
        74, 122, 129, 152, 118, 112, 110,  27,  54,  60,  95,  98, 106,
        94,  94,  88,  85,  93,  88,  29,  27,  40,  83,  98,  93,  77,
        66, 100,  78,  83,  69,  32,  27,  39,  74,  88,  99,  72,  59,
        74,  68,  80,  90,  20,  36,  50,  70,  81, 114,  58,  54,  79,
        57,  73,  83,  19,  23,  53,  66,  99,  97,  87,  96, 104,  85,
        93, 110,  23,  42,  40,  34,  53,  61,  91,  77,  84,  77,  79,
        83,   5,   3,   6,   6,  11,  16,  13,  25,  26,  13,  27,  19,
        10,  10,  14,  37,  42,  58,  13,  17,  13,  19,  12,  17,  28,
        48,  92, 129, 131, 139, 125, 123, 124, 126, 141, 118,  2

In [82]:
volume_prediction

array([ 12.484     ,  16.2       ,  19.844     ,  26.156     ,
        28.204     ,  40.884     ,  47.388     ,  38.068     ,
        34.2       ,  50.766     ,  43.016     ,  48.532     ,
        17.41      ,  15.712     ,  16.702     ,  21.574     ,
        34.878     ,  38.848     ,  39.106     ,  33.386     ,
        46.462     ,  39.408     ,  39.372     ,  35.588     ,
        14.448     ,  15.322     ,  20.726     ,  21.666     ,
        29.792     ,  36.024     ,  39.098     ,  39.866     ,
        40.286     ,  43.786     ,  35.77      ,  37.038     ,
        14.502     ,  12.816     ,  19.226     ,  26.216     ,
        38.27      ,  37.234     ,  44.136     ,  45.762     ,
        49.274     ,  50.866     ,  54.182     ,  56.958     ,
        15.66      ,   8.186     ,  19.308     ,  15.154     ,
        25.296     ,  32.132     ,  36.784     ,  44.176     ,
        47.298     ,  44.582     ,  50.008     ,  48.752     ,
        17.822     ,  10.242     ,   8.374     ,  16.19